Now that we understand the indexing pipeline, we can
utilize our vector database to retrieve relevant documents
for a given query and use them to generate a response.

In [1]:

# load the environment variables
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

# lets just validate that we have the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

loader = WebBaseLoader("https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:

# setup the chat model and prompt template
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about the US Constitution. Use the provided context to answer the question. IMPORTANT: If you are unsure of the answer, say 'I don't know' and don't make up an answer."),
    ("user", "Question: {question}\nContext: {context}")
])

chain = prompt | llm

In [4]:

query = "Is there any plans to change the US Constitution to allow for term limits on senators?"

# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs})

print(response.content)

I don't know.


In [5]:
while True:
    query = input("Enter a question: ")
    if query == "exit":
        break
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})
    print(response.content)

The U.S. Constitution originally consists of seven articles.
The Constitution states that the President may grant reprieves and pardons for offenses against the United States, except in cases of impeachment. This is found in Article II, Section 2, Clause 1 of the Constitution.
